# User features

## Import packages

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import random
random.seed(32)
import gensim
from gensim import corpora, models
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

## Import data

In [2]:
# authors = pd.read_csv('/home/sophia/ma_py/author_profiles.csv')

df = pd.read_pickle("preprocessed_author.pkl")
# df = pre_df.set_index(['index', 'author'])
# df.head(10)
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422 entries, 0 to 421
Data columns (total 477 columns):
 #    Column                 Dtype  
---   ------                 -----  
 0    index                  int64  
 1    author                 object 
 2    complete_body          object 
 3    doc_body               object 
 4    probody                object 
 5    tokens                 object 
 6    senttokens             object 
 7    agreeableness          float64
 8    openness               float64
 9    conscientiousness      float64
 10   extraversion           float64
 11   neuroticism            float64
 12   agree                  int64  
 13   openn                  int64  
 14   consc                  int64  
 15   extra                  int64  
 16   neuro                  int64  
 17   all_utc                object 
 18   mean_controversiality  float64
 19   mean_gilded            float64
 20   num_subreddits         int64  
 21   all_lang               int64  
 22   

In [3]:
df.tail()

,index,author,complete_body,doc_body,probody,tokens,senttokens,agreeableness,openness,conscientiousness,...,fantasyfootball,nostalgia,watchpeopledie,homebrewing,nexus6p,canadapolitics,rottweiler,ipad,samplesize,starwarsbattlefront
417,424,xanthraxoid,I'd really like this video to include some inf...,[I'd really like this video to include some in...,[would really like video include information c...,"[[would, really, like, video, include, informa...",[[I'd really like this video to include some i...,86.0,45.0,8.0,...,0,0,0,0,0,0,0,0,0,0
418,425,xenomouse,"You're a guy, aren't you? I can definitely see...","[You're a guy, aren't you? I can definitely se...",[guy not definitely see would make boy scene f...,"[[guy, not, definitely, see, would, make, boy,...","[[You're a guy, aren't you?, I can definitely ...",26.0,93.0,49.0,...,0,0,0,0,0,0,0,0,0,0
419,426,xeroctr3,man even the thought of it makes me depressed....,[man even the thought of it makes me depressed...,[man even thought makes depressed loving someo...,"[[man, even, thought, makes, depressed, loving...",[[man even the thought of it makes me depresse...,3.0,75.0,27.0,...,0,0,0,0,0,0,0,0,0,0
420,427,xzack18,Not all of us are out to kill,[Not all of us are out to kill],[not us kill],"[[not, us, kill]]",[[Not all of us are out to kill]],4.0,19.0,11.0,...,0,0,0,0,0,0,0,0,0,0
421,428,zugzwang_03,Institutions should accommodate religious or s...,[Institutions should accommodate religious or ...,[institutions accommodate religious serious me...,"[[institutions, accommodate, religious, seriou...",[[Institutions should accommodate religious or...,10.0,41.0,86.0,...,0,0,0,0,0,0,0,0,0,0


## LDA

In [4]:
# https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html

# problem: probody does not have punctuation anymore
# multicore instead of lda model seems to be slower????

def apply_lda(df, number, name):
    print("Preprocessing for LDA...")
    neglst = ["no", "not", "none", "nobody", "nothing", "neither", "nowhere", "never", "nay"]
    inputlst = []
    for row in tqdm(df['tokens']):
        rowlst = []
        for comment in row:
            rowlst.append([word for word in comment if (word not in neglst)])
        inputlst.append(rowlst)
#     df['ldatokens'] = df['tokens'].apply(lambda x: [word for row in x for word in row if (word not in negationsList)])
    # filter out stopwords including negations for topic modeling
#     stopwordList = stopwords.words('english')
#     df['ldatokens'] = df['probody'].apply(lambda x: ' '.join([x for x in x.split() if (x not in stopwordList)]))
#     df['ldatokens'] = df['ldatokens'].apply(lambda x: (word_tokenize(x)))
#     print(df.iloc[421]['ldatokens'])
    print("Start LDA...")
    lst = []
    for row in tqdm(inputlst):
        if len(row) < 2:
            lst.append(-1)
        else:
            dictionary = corpora.Dictionary(row)
            corpus = [dictionary.doc2bow(text) for text in row]
            ldamodel = gensim.models.LdaMulticore(corpus, num_topics=number, id2word = dictionary, passes=20, workers=15)
            result = ldamodel.print_topics(num_topics=1, num_words=1)
#             result = ldamodel.get_document_topics(corpus)
            res = list(result)
            topic = [item[0] for item in res]
            lst.append(topic[0])
    df[name] = lst
    return df

# print(df.iloc[220]['tokens'])
# print(len(df.iloc[220]['tokens']))
# df = apply_lda(df, 50, "ldafifty")
# df['ldathree']

# Wrapper

In [5]:
def extract_userfeatures(df):
    print("LDA with fifty topics: ")
    df = apply_lda(df, 50, "ldafifty")
    print("LDA with onehundred topics: ")
    df = apply_lda(df, 100, "ldahundred")
    return df

In [6]:
user_feat_df = extract_userfeatures(df)

Preprocessing for LDA...


  0%|          | 0/422 [00:00<?, ?it/s]

Start LDA...


  0%|          | 0/422 [00:00<?, ?it/s]

Preprocessing for LDA...


  0%|          | 0/422 [00:00<?, ?it/s]

Start LDA...


  0%|          | 0/422 [00:00<?, ?it/s]

In [8]:
user_feat_df.to_pickle("user_feat_df.pkl")